## FusionBase_UNODC generic script:

In [1]:
# Networking
import requests

# File handling
import zipfile
import os
import json

# Data wrangling
import pandas as pd
import re
import numpy as np
import hashlib

from collections import Counter

# Dateteime
from datetime import datetime
#update
from apscheduler.schedulers.blocking import BlockingScheduler

#Schedule the automated tasks to be done regularly for all possible data sources
def update():
    #Downloading data from sources
    url=['car_theft.xlsx','theft_0.xlsx','bulglary.xlsx','money_laundering.xlsx','fraud.xlsx','cjs_court.xlsx','cjs_prosecution.xlsx','cjspolicepersonnel.xlsx','cjsp_convicted.xlsx','cjsp_prosecuted.xlsx','cjsp_fromal_contact.xlsx','prisionfacilities.xlsx','prison_capacity.xls','prisonstaff.xlsx','personsenteringprisons.xlsx','personheld-mortality.xlsx','personsheld-unsentenced-length.xlsx','unsentenced.xlsx',
'total_unsentenced.xls','personheld-nationality.xlsx','personsheld.xlsx','persons_held_total.xls','peopleconvicted-tip.xlsx','victimcitizens.xlsx','detected-trafficking.xlsx','sexual_exploitation_0.xlsx','sexual_violence_0.xlsx','kidnapping_0.xlsx','robbery_0.xlsx','serious_assault_0.xlsx','homicide_sex_citizenship.xls','intentional_homicide-bysexandage.xls','city-homicide.xls','homicide-citizenship.xls','crime_related_situational_context.xls','ipfm.xls','homicidebyagesex.xlsx','armsseizedfoundsurrenderedbytracing.csv_.xlsx','armsseizedbytype.xlsx','wholesales-price.xlsx','seizure-table.xlsx','treatment.xlsx','drugrelateddeath.xlsx','pwid-hiv.xlsx']       
    try:
        for u in url:
            r = requests.get('https://dataunodc.un.org/sites/dataunodc.un.org/files/'+u, allow_redirects=True)  
            open('C:/Users/Adem/Desktop/FusionBase/'+u, 'wb').write(r.content)
    except Exception:
        print('Could not download data') 

    #Extracting data from excel files
    for u in url:
        df = pd.read_excel('C:/Users/Adem/Desktop/FusionBase/'+u, index_col=None, header=None)
        #Calculating the number of columns and indexes and stock them in lists to distinguish columns and indexes 
        #by trying to measure the length of distinct values and compare it to the length of columns

        col_count=0
        ind_count=0
        AllList=[[] for i in range(len(df))]
        columnsList=[[] for i in range(len(df))]
        indexList=[[] for i in range(len(df))]
        for j in range(len(df.columns)):
            for i in range(len(df)):
        
                if((type(df.iloc[i,j])!=int and type(df.iloc[i,j])!=float and str(df.iloc[i,j])!='nan' 
                    and type(df.iloc[i,j])!=np.float64) or (i==0 and str(df.iloc[i,j])!='nan')):
            
                        AllList[i].append(df.iloc[i,j])
                
                
                
                if((type(df.iloc[i,j])!=int and type(df.iloc[i,j])!=float and str(df.iloc[i,j])!='nan')
                   and type(df.iloc[i,j])!=np.float64):
                        columnsList[j].append(df.iloc[i,j])
                        if (len(columnsList[j])>len(df)/2 ):
                            if (0<=i<len(df)-1):
                                if (type(df.iloc[i+1,j])!=int and type(df.iloc[i+1,j])!=float):
                                    col_count=j+1
                        
                            else: 
                                col_count=j+1
                       
                        indexList[i].append(df.iloc[i,j])
                        if (len(indexList[i])>len(df.columns)/2):
                            ind_count=i+1
        #determine the indexes,the additional indexes, the columns and their names
        if ind_count==0:
            ind_count=1        
        names=[]
        additionalindex=[]
        columns=[]
        indexList=AllList[col_count:]
        for i in range(ind_count):
            if(i==0):
        
                if(len(AllList[i])==len(df.columns)):
                    additionalindex.append(list(Counter(AllList[0]).keys())[col_count:])
                    names.append(AllList[i][:col_count])
            
                else:
                    additionalindex.append(list(Counter(AllList[0]).keys()))
            
            elif(i==ind_count-1 and len(AllList[0])!=len(df)):
                columns.append(AllList[i][col_count:])
                names.append(AllList[i][:col_count])
            elif(1<len(Counter(df.iloc[i]).keys()) <4 ):
                additionalindex.append(AllList[i])

        tuples=AllList[ind_count:]
        tuplesfinal=[]
        for i in range(len(tuples)):
            for j in range(len(additionalindex)):
                for k in range(len(additionalindex[j])):
                    tuplesfinal.append(tuples[i]+[additionalindex[j][k]])
            
        namesfinal=[]
        for i in range(len(names)):
            for j in range(len(additionalindex)):
                namesfinal.append(names[i]+[j])
        columnsFinal=[]
        if columns:
            columnsFinal=list(Counter(columns[0]).keys())
        #determine the values to be diplayed
        values=[]
        if columnsFinal:
            for i in range(ind_count,len(df)):
                for j in range(col_count,len(df.columns)-1,len(columnsFinal)):
                    row=[]
                    for k in range(len(columnsFinal)):
                        row.append(df.iloc[i,j+k])
                    values.append(row)
        else:
            for i in range(ind_count,len(df)):
                for j in range(col_count,len(df.columns),1):
                    values.append(df.iloc[i,j])           
        #transform the data to a pivot table
        index = pd.MultiIndex.from_tuples(tuple(map(tuple,tuplesfinal)), names=namesfinal[0])
         
        if columns:
    
            df = pd.DataFrame(values, index=index, columns=list(Counter(columns[0]).keys()))
        else:
            df = pd.DataFrame(values, index=index)
        date_time=[]
        #Adding the datetime and country_iso_alpha3
        for i in range(len(df)):
            date_time.append(datetime.now())
        df['fb_datetime']=date_time
        alpha=[]
        alpha3_lookups=None
        if alpha3_lookups == None:
            with open('C:/Users/Adem/Desktop/FusionBase/Fusion/alpha3_lookups.txt', 'r') as jfile:
                data=jfile.read()
                alpha3_lookups = json.loads(data)
        
        df=df.reset_index()
        for i in range(len(df['Country'])):
            print(type(df['Country'][i]))
            name=df['Country'][i]
            name = name.strip().lower()
            name = hashlib.sha256(name.encode('utf-8')).hexdigest()

            if name in alpha3_lookups:
                alpha.append(alpha3_lookups[name])
        
            else:
                alpha.append('None')
        df['country_iso_alpha3']=alpha 
        #store the transformed data in destination
        df.columns = df.columns.astype(str)
        df.to_parquet('C:/Users/Adem/Desktop/FusionBase/parquet/'+u,compression='gzip',engine='fastparquet')

scheduler = BlockingScheduler()
scheduler.add_job(update, 'interval', minutes=1)
scheduler.start()


Execution of job "update (trigger: interval[0:01:00], next run at: 2020-11-04 23:21:26 CET)" skipped: maximum number of running instances reached (1)
Job "update (trigger: interval[0:01:00], next run at: 2020-11-04 23:22:26 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Adem\Anaconda3\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-1-c1c979ff71c8>", line 127, in update
    with open('C:/Users/AhmedMasmoudi/Desktop/FusionBase/Fusion/alpha3_lookups.txt', 'r') as jfile:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/AhmedMasmoudi/Desktop/FusionBase/Fusion/alpha3_lookups.txt'
Execution of job "update (trigger: interval[0:01:00], next run at: 2020-11-04 23:23:26 CET)" skipped: maximum number of running instances reached (1)
Execution of job "update (trigger: interval[0:01:00], next run at: 2020-11-04 23:24:26 CET)" skipped: maximum number of running 

Traceback (most recent call last):
  File "C:\Users\Adem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-c1c979ff71c8>", line 149, in <module>
    scheduler.start()
  File "C:\Users\Adem\Anaconda3\lib\site-packages\apscheduler\schedulers\blocking.py", line 19, in start
    self._main_loop()
  File "C:\Users\Adem\Anaconda3\lib\site-packages\apscheduler\schedulers\blocking.py", line 28, in _main_loop
    self._event.wait(wait_seconds)
  File "C:\Users\Adem\Anaconda3\lib\threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "C:\Users\Adem\Anaconda3\lib\threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in show

KeyboardInterrupt: 

In [ ]:
car_theft.xlsx,theft_0.xlsx,bulglary.xlsx,money_laundering.xlsx,fraud.xlsx,cjs_court.xlsx,cjs_prosecution.xlsx,cjspolicepersonnel.xlsx,cjsp_convicted.xlsx,cjsp_prosecuted.xlsx,cjsp_fromal_contact.xlsx,prisionfacilities.xlsx,prison_capacity.xls,prisonstaff.xlsx,personsenteringprisons.xlsx,personheld-mortality.xlsx,personsheld-unsentenced-length.xlsx,unsentenced.xlsx,
total_unsentenced.xls,personheld-nationality.xlsx,personsheld.xlsx,persons_held_total.xls,peopleconvicted-tip.xlsx,victimcitizens.xlsx,detected-trafficking.xlsx,sexual_exploitation_0.xlsx,sexual_violence_0.xlsx,kidnapping_0.xlsx,robbery_0.xlsx,serious_assault_0.xlsx,homicide_sex_citizenship.xls,intentional_homicide-bysexandage.xls,city-homicide.xls,homicide-citizenship.xls,crime_related_situational_context.xls,ipfm.xls,homicidebyagesex.xlsx,armsseizedfoundsurrenderedbytracing.csv_.xlsx,armsseizedbytype.xlsx,wholesales-price.xlsx,seizure-table.xlsx,treatment.xlsx,drugrelateddeath.xlsx,pwid-hiv.xlsxcar_theft.xlsx,theft_0.xlsx,bulglary.xlsx,money_laundering.xlsx,fraud.xlsx,cjs_court.xlsx,cjs_prosecution.xlsx,cjspolicepersonnel.xlsx,cjsp_convicted.xlsx,cjsp_prosecuted.xlsx,cjsp_fromal_contact.xlsx,prisionfacilities.xlsx,prison_capacity.xls,prisonstaff.xlsx,personsenteringprisons.xlsx,personheld-mortality.xlsx,personsheld-unsentenced-length.xlsx,unsentenced.xlsx,
total_unsentenced.xls,personheld-nationality.xlsx,personsheld.xlsx,persons_held_total.xls,peopleconvicted-tip.xlsx,victimcitizens.xlsx,detected-trafficking.xlsx,sexual_exploitation_0.xlsx,sexual_violence_0.xlsx,kidnapping_0.xlsx,robbery_0.xlsx,serious_assault_0.xlsx,homicide_sex_citizenship.xls,intentional_homicide-bysexandage.xls,city-homicide.xls,homicide-citizenship.xls,crime_related_situational_context.xls,ipfm.xls,homicidebyagesex.xlsx,armsseizedfoundsurrenderedbytracing.csv_.xlsx,armsseizedbytype.xlsx,wholesales-price.xlsx,seizure-table.xlsx,treatment.xlsx,drugrelateddeath.xlsx,pwid-hiv.xlsx